In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import PIL

In [ ]:
# Return new image
def read_write_image(img_name):
    img = Image.open(img_name)
    ext = input('Enter extension: ')
    return img.save(img.filename.split('.')[0] + '.' + ext)

In [ ]:
def show_image(img_list):
    dpi = plt.rcParams['figure.dpi']
    plt.figure(figsize=(img_list[0].size[0]/dpi, img_list[0].size[1]/dpi))
    [plt.subplot(1, len(img_list), i+1) and plt.imshow(img_list[i]) for i in range(len(img_list))]
    plt.show()

img_list = [
    Image.open('1.png'),
    Image.open('2.png')
]

show_image(img_list)

# B1: Tìm k centroids
## Vấn đề: Random không lấy lại màu cũ
# B2: Tính khoảng cách từ k centroids đến các điểm dữ liệu
## Sự thay đổi của centroid là không đáng kể hoặc không nhiều hoặc max_iter

### Màu cuối cùng tìm được không nhất thiết phải có trong ảnh
### Mask numpy